In [ ]:
import os
import sys
import pymongo
import pandas as pd
import plotly.express as px
import csv
import datetime
from datetime import datetime
import re
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
load_dotenv()
# Replace with your MongoDB connection string
database_url = os.getenv('db_url_for_creating')

# Connect to the MongoDB server
client = MongoClient(database_url)

# Access the specific database
db_name = os.getenv('db_name')
db = client[db_name]

three_weeks_ago = datetime.now() - timedelta(days=10) #datetime.now() - timedelta(weeks=3)

now = datetime.now()

# Calculate start and end of today
start_of_today = datetime(now.year, now.month, now.day)  # 00:00:00 today
end_of_today = start_of_today + timedelta(days=3)  # 00:00:00 tomorrow

# Calculate start of yesterday
start_of_yesterday = start_of_today - timedelta(days=20)
print(start_of_yesterday)
print(end_of_today)

# Iterate over all collections in the database
for collection_name in db.list_collection_names():
    collection = db[collection_name]
    
    # Assuming the documents have a field named 'created_at' or similar which stores the date
    # Replace 'created_at' with the appropriate field name in your collections
    result = collection.delete_many({"createdAt": {'$gte': start_of_yesterday,'$lt': end_of_today}})
    
    print(f"Deleted {result.deleted_count} documents from {collection_name}")

print("Data deletion complete.")

2024-11-14 00:00:00
2024-12-07 00:00:00
Deleted 0 documents from comments
Deleted 0 documents from commentdislikes
Deleted 0 documents from postdislikes
Deleted 15 documents from users
Deleted 0 documents from idstorages
Deleted 0 documents from posts
Deleted 0 documents from subscriptions
Deleted 0 documents from sampleCollection
Deleted 0 documents from commentlikes
Deleted 0 documents from viewposts
Deleted 0 documents from readposts
Deleted 0 documents from reposts
Deleted 0 documents from postlikes
Deleted 0 documents from selectedusers
Deleted 0 documents from users5
Data deletion complete.


In [ ]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import os

# Replace with your MongoDB connection string
database_url = os.getenv('db_url_for_creating')

# Connect to the MongoDB server
client = MongoClient(database_url)

db = client["test_llm_1"]
users_collection = db["users"]     # Replace with your collection name

# Define the cutoff date
cutoff_date = datetime(2024, 11, 20, 9, 9, 59)

# Delete users created after the cutoff date
result = users_collection.delete_many({"createdAt": {"$gt": cutoff_date}})
print(f"Deleted {result.deleted_count} documents.")

Deleted 15 documents.


In [ ]:

source_db = client["ms1"]  # Source database
target_db = client["ms3"]  # Target database

# Copy all documents from the "users" collection
users = list(source_db["users"].find())
if users:
    target_db["users"].insert_many(users)
    print(f"Copied {len(users)} users to ms4.")

# Copy only the first 20 documents from the "posts" collection
posts = list(source_db["posts"].find().limit(20))
if posts:
    target_db["posts"].insert_many(posts)
    print(f"Copied {len(posts)} posts to ms4.")

Copied 15 users to ms4.
Copied 20 posts to ms4.


In [ ]:
from pymongo import MongoClient
import os

# Replace with your MongoDB connection string
database_url = os.getenv('db_url_for_creating')

# Connect to the MongoDB server
client = MongoClient(database_url)

db = client["fake_news"]
posts_collection = db["posts"]  # Replace with your collection name

# Find the first 150 documents based on the creation order
first_150_docs = posts_collection.find().sort("createdAt", 1).limit(150)

# Store the IDs of these documents
first_150_ids = [doc["_id"] for doc in first_150_docs]

# Delete all documents that are not in the first 150
result = posts_collection.delete_many({
    "_id": {"$nin": first_150_ids}
})

print(f"Deleted {result.deleted_count} documents.")



Deleted 3 documents.


In [ ]:
from pymongo import MongoClient
from datetime import datetime

# Connect to MongoDB
database_url = os.getenv('db_url_for_creating')
client = MongoClient(database_url)  # Adjust the URI if needed
db = client["test_llm_1"]  # Replace with your database name

collections = ["commentlikes", "comments", "postlikes", "posts"]

# Initialize variables to track earliest and latest dates
earliest = {"collection": None, "createdAt": None}
latest = {"collection": None, "createdAt": None}

for collection_name in collections:
    collection = db[collection_name]
    
    # Find the document with the earliest and latest `createdAt` values
    first_doc = collection.find_one(sort=[("createdAt", 1)])
    last_doc = collection.find_one(sort=[("createdAt", -1)])
    
    # Update the earliest
    if first_doc and (earliest["createdAt"] is None or first_doc["createdAt"] < earliest["createdAt"]):
        earliest = {"collection": collection_name, "createdAt": first_doc["createdAt"]}
    
    # Update the latest
    if last_doc and (latest["createdAt"] is None or last_doc["createdAt"] > latest["createdAt"]):
        latest = {"collection": collection_name, "createdAt": last_doc["createdAt"]}

# Calculate the time difference
if earliest["createdAt"] and latest["createdAt"]:
    time_diff = latest["createdAt"] - earliest["createdAt"]
    print(f"Earliest: {earliest['createdAt']} in collection {earliest['collection']}")
    print(f"Latest: {latest['createdAt']} in collection {latest['collection']}")
    print(f"Time difference: {time_diff}")
else:
    print("Could not determine earliest or latest createdAt.")

# Close the connection
client.close()


Earliest: 2024-11-22 10:36:24.906000 in collection posts
Latest: 2024-11-22 12:23:59.337000 in collection posts
Time difference: 1:47:34.431000
